## 1. Load Data

In [15]:
# convert text data in the txt file into a pandas dataframe

f = open("4U_Reviews.txt", "r", encoding = "ISO-8859-1")

#map the content to dictionary
dict_review = {'title':[], 'user_name':[], 'country':[], 'date':[],
              'rating':[], 'review':[], 'type_traveller':[], 'cabin':[], 
               'route_from':[], 'route_to':[], 'date_flown':[], 'recommend':[]}

review_info_list = []
for line in f:
    if line == '\n':
        print("-----")
        #print(review_info_list)
        for i, line in enumerate(review_info_list):
            if i == 0:
                #print('title: {}'.format(line))
                dict_review['title'].append(line)
            elif i == 1:
                pos_start = line.find('(')
                pos_end = line.find(')')
                user_name = line[0:pos_start].strip()
                country_name = line[pos_start+1:pos_end].strip()
                date_str = line[pos_end+1:].strip()
                #print('user name: {}'.format(user_name))
                #print('country name: {}'.format(country_name))
                #print('date: {}'.format(date_str))
                dict_review['user_name'].append(user_name)
                dict_review['country'].append(country_name)
                dict_review['date'].append(date_str)
                review_info_list = []
            elif i == 2:
                rating = int(line.strip())
                print('rating: {}'.format(rating))
                
    else:
        line = line.replace('\"', '')
        #print(line)
        review_info_list.append(line)
    
    
            
            
        

-----
rating: 2
-----
rating: 3
-----
rating: 3
-----
rating: 10
-----
rating: 4
-----
rating: 1
-----
rating: 7
-----
rating: 2
-----
rating: 2
-----
rating: 5
-----
rating: 2
-----
rating: 10
-----
rating: 1
-----
rating: 2
-----
rating: 7
-----
rating: 7
-----
rating: 9
-----
rating: 2
-----
rating: 4
-----
rating: 7
-----
rating: 8
-----
rating: 10
-----
rating: 2
-----
rating: 9
-----
rating: 8
-----
rating: 7
-----
rating: 8
-----
rating: 2
-----
rating: 10
-----
rating: 2
-----
rating: 10
-----


ValueError: invalid literal for int() with base 10: 'na'

## 2. EDA

## 3. Modelling